In [1]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re
pd.set_option('display.max_columns', None)

In [2]:
# location of raw data files
input_path = "../../datasets/raw/"

In [3]:
#read PROCEDUREEVENTS_MV.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'NOTEEVENTS.csv',dtype = "object")

In [4]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
ROW_ID         2083180 non-null object
SUBJECT_ID     2083180 non-null object
HADM_ID        1851344 non-null object
CHARTDATE      2083180 non-null object
CHARTTIME      1766614 non-null object
STORETIME      1246404 non-null object
CATEGORY       2083180 non-null object
DESCRIPTION    2083180 non-null object
CGID           1246404 non-null object
ISERROR        886 non-null object
TEXT           2083180 non-null object
dtypes: object(11)
memory usage: 174.8+ MB


In [5]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [6]:
#Dropping rows that have error data
df = df[df['ISERROR']!='1'].reset_index(drop = True)

In [7]:
# Replacing missing values for STORTIME with CHARTDATE
df.loc[df['STORETIME'].isnull(),'STORETIME']=df['CHARTDATE']
df['STORETIME'] = pd.to_datetime(df['STORETIME'])

In [8]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,2151-08-04,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,2118-06-14,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,2119-05-25,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,2124-08-18,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,2162-03-25,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [9]:
df = df.drop(labels=['CHARTDATE','CHARTTIME','CGID','ISERROR'], axis = 'columns')

In [10]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,STORETIME,CATEGORY,DESCRIPTION,TEXT
0,174,22532,167853,2151-08-04,Discharge summary,Report,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,Discharge summary,Report,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,Discharge summary,Report,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,Discharge summary,Report,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,Discharge summary,Report,Admission Date: [**2162-3-3**] D...


In [11]:
%%time
#Cleaning the data by removing special charecters and masked data
df['TEXT'] = df['TEXT'].map(lambda x: re.sub('\[.*?\]|[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', ' ', x)).copy()

CPU times: user 1min 58s, sys: 2.66 s, total: 2min
Wall time: 2min


In [12]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,STORETIME,CATEGORY,DESCRIPTION,TEXT
0,174,22532,167853,2151-08-04,Discharge summary,Report,Admission Date: Discharge Date: ...
1,175,13702,107527,2118-06-14,Discharge summary,Report,Admission Date: Discharge Date: D...
2,176,13702,167118,2119-05-25,Discharge summary,Report,Admission Date: Discharge Date...
3,177,13702,196489,2124-08-18,Discharge summary,Report,Admission Date: Discharge Date...
4,178,26880,135453,2162-03-25,Discharge summary,Report,Admission Date: Discharge Date...


In [13]:
# Create Notevents
df['noteevents'] = df['CATEGORY']+' '+df['DESCRIPTION']+' '+df['TEXT']

In [14]:
df = df.drop(labels=['CATEGORY','DESCRIPTION','TEXT'], axis = 'columns')

In [15]:
# not using a map function as it occupies a lot of memory
from tqdm.auto import tqdm
for i in tqdm(range(len(df))):
    df.at[i, ('noteevents')] = df.at[i, ('noteevents')].split()

In [16]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,STORETIME,noteevents
0,174,22532,167853,2151-08-04,"[Discharge, summary, Report, Admission, Date:,..."
1,175,13702,107527,2118-06-14,"[Discharge, summary, Report, Admission, Date:,..."
2,176,13702,167118,2119-05-25,"[Discharge, summary, Report, Admission, Date:,..."
3,177,13702,196489,2124-08-18,"[Discharge, summary, Report, Admission, Date:,..."
4,178,26880,135453,2162-03-25,"[Discharge, summary, Report, Admission, Date:,..."


In [17]:
%%time
df['STORETIME'] = df['STORETIME'].map(lambda x: str(x))

CPU times: user 50.8 s, sys: 372 ms, total: 51.2 s
Wall time: 50.8 s


In [18]:
# create output path
mypath_output = "../../datasets/noteevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [19]:
# export the dataframe to JSON format
df.to_json(mypath_output+"noteevents.json",orient = 'records') 

In [20]:
df['len'] = df['noteevents'].map(lambda x: len(x))

In [22]:
df['len'].describe()

count    2.082294e+06
mean     2.400073e+02
std      3.294293e+02
min      2.000000e+00
25%      6.400000e+01
50%      1.400000e+02
75%      2.610000e+02
max      7.613000e+03
Name: len, dtype: float64

In [23]:
sum(df['len'])

499765847

In [24]:
len(df)

2082294